#### PROYECTO INDIVIDUAL NUMERO 2
##### Nicolas Gallucci

importacion de librerias necesarias para la etapa de carga y preprocesamiento de datos.

In [4]:
import pandas as pd
import numpy as np
import sys
import requests
import re
from bs4 import BeautifulSoup 

1) Importamos las bases de datos de entrenamiento y testeo.

In [5]:
train = pd.read_csv ("Datathon\properties\properties_colombia_train.csv")
test = pd.read_csv ("Datathon\properties\properties_colombia_test.csv")

2) Seteo de configuracion necesaria dentro de pandas.

In [482]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',100)
pd.set_option('display.float_format', '{:.2f}'.format)


3) Analisis exploratorio de los datos con Profile Report.

In [ ]:
from pandas_profiling import ProfileReport
ProfileReport(train)

4) Normalizacion de valores vacios cambiandolos por el valor de "nan" para lograr una tranformacion mas rapida y precisa de las columnas.

In [6]:
train.replace(r'^\s*$', np.nan, regex=True,inplace=True)
test.replace(r'^\s*$', np.nan, regex=True,inplace=True)

5. Pequeño scraping para obtener el valor de la divisa colombiana en dolares para poder completar los valores que estan en dicha moneda dentro del dataset.

In [9]:
url = "https://www.capitalcolombia.com/index.php?sec=trm_precio_dolar_en_colombia&pag=ano&consulta=2020"
page = requests.get(url)
soup = BeautifulSoup(page.content,"html.parser")
precio = soup.find_all("table",class_="table table-striped table-bordered col-10")
#busco dentro del url con ayuda de request y soup, el codigo html que me brinda la informacion del registro diario del año 2020 sobre el precio del dolar en Colombia.

x = str(precio).split()
x = re.findall(r'align="center">\S*[^0-9]',str(x).replace(",",""))
peso_colombiano = re.findall(r'[$]+\d......',str(x))
fecha = re.findall(r'[^$]\d....2020',str(x))

#Utilizo Regex para filtrar los datos que necesito y asi crear el dataframe.

tabla = pd.DataFrame({
        "peso_colombiano":peso_colombiano,
        "start_date":fecha
        })

tabla['start_date'] = pd.to_datetime(tabla['start_date'], format="%d/%m/%Y")
tabla['start_date'].replace("%d/%m/%Y","%Y/%m/%d",inplace=True)
tabla['start_date'] = tabla['start_date'].astype(str)

tabla["peso_colombiano"] = tabla["peso_colombiano"].str.replace("$","")
tabla['peso_colombiano'] = tabla['peso_colombiano'].astype(float)

#Configuro la tabla para que las fechas me queden en el mismo orden que el dataset de entrenamiento, de esa manera, en los proximos pasos
#podre concadenarlas y normalizar los precios dolarizados dentro del set de datos.

C:\Users\nicol\AppData\Local\Temp\ipykernel_8996\675915861.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tabla["peso_colombiano"] = tabla["peso_colombiano"].str.replace("$","")


6) Columnas a eliminar (explicado)

In [7]:
train.drop(columns=["Unnamed: 0","end_date","id","ad_type","l1","l4","l5","l6","surface_total",
                        "surface_covered","title","description","rooms","bedrooms","price_period",
                        "operation_type","created_on","geometry"],inplace=True)
                        
test.drop(columns=["Unnamed: 0","end_date","id","ad_type","l1","l4","l5","l6","surface_total",
                        "surface_covered","title","description","rooms","bedrooms","price_period",
                        "operation_type","created_on","geometry","currency","start_date"],inplace=True)


# Las 3 razones mas importantes por las cuales elimine estas columnas a priori, fue porque la mayoria contaban con mas del 80% de valores faltantes (surface_total,rooms,bedrooms,l6,l5,l4...).
# Otras eran columnas que solo ofrecian informacion muy variada y categorica que me iba a ser imposible modelar (description,title,...),
# y el resto por ser columnas sin informacion relevante, o ser redundantes para el modelo (Unnamed: 0,id,ad_type,created_on...)

7) Luego del analisis exploratorio y teniendo en mente que columnas voy a dropear y porque, vamos a completar aquellas columnas con valores faltantes.

In [ ]:
#COLUMNA DE PRECIOS

train["currency"].value_counts() 
#-- 8 valores en USD, son insignificantes en comparacion al dataset recibido de entrenamiento, 
# pero podria ser una buena practica si el dataset en un futuro incrementara, y asi los casos tambien de viviendas en dolares.


train = pd.merge(train, tabla, on='start_date')
train["price"][train["currency"] == "USD"] = train["price"] * train["peso_colombiano"] 
train.drop(columns=["peso_colombiano","currency","start_date"],inplace=True)           
#-- Normalizo a precio colombianos, combinando las tablas "train" y "tabla" y luego dropeando las columnas                                                                                        
# "peso_colombiano" , "currency" (ya que esta ultima es redundante) y "start_date" que no me servira para para modelo de clasificacion.


train["price"].isna().sum() #--- 63 valores faltantes
train["price"][train["price"].isna()] = train["price"].mean() #-- Completo los faltantes con el promedio de todos los precios.

In [ ]:
#COLUMNA DE BAÑOS

promedios = []  

for i in range(len(train.property_type.value_counts().index)):  
    tipo = train.property_type.value_counts().index[i],train.bathrooms [train.property_type == train.property_type.value_counts().index[i]].mean()
    promedios.append(tipo)
#-- Con este loop consigo el promedio de baños por tipo de propiedad dentro de la lista "promedios", de esta manera podre continuar aquellas 
# propiedades con baños faltantes de forma mas precisa.

for i in range (len(promedios)): #-- En cada instancia completo la columna de baños faltantes con su respectivo promedio.
    train.bathrooms[(train.bathrooms.isna()) & (train.property_type == promedios[i][0])] = promedios[i][1]

train["bathrooms"][train["bathrooms"].isna()] = 0 #-- Aquelas filas de la columna baño que no pudieron ser completadas al ser "nan" las normalizo con 0.
train.bathrooms = train.bathrooms.astype(int) #-- Para no tener decilamales, uso "int" para tener solo numeros enteros.


#----------------------------------------------------------------------------------------------------------------------------------------------------
#APLICADO AL DATASET DE TEST

promedios_t = []  

for i in range(len(test.property_type.value_counts().index)):  
    tipo = test.property_type.value_counts().index[i],test.bathrooms [test.property_type == test.property_type.value_counts().index[i]].mean()
    promedios_t.append(tipo)

for i in range (len(promedios_t)): 
    test.bathrooms[(test.bathrooms.isna()) & (test.property_type == promedios_t[i][0])] = promedios_t[i][1]

test["bathrooms"][test["bathrooms"].isna()] = 0 

test.bathrooms = test.bathrooms.astype(int) 

In [ ]:
#COMPLETANDO COLUMNA DE L3 PARA POSTERIOR COMPLETADO DE "LAT" Y "LON"

localilades_na = train["l2"][train.l3.isna()].drop_duplicates().values 
# Lista de "l2" que poseen "l3" con nulos, usando la moda de "l3" con el filtro de "l2",                                                                   
# se puede conseguir un valor mas preciso para completar los registros de "l3" faltantes que sean a.                                                        
for i in localilades_na:
    if len(train["l3"][train["l2"] == i].mode()) > 0: #-- condicion por si la moda devuelve nulo y evitar que el loop de error.
        train["l3"][(train["l2"] == i) & (train["l3"].isna())] = train["l3"][train["l2"] == i].mode()[0]


train["l3"][train["l3"].isna()] = "sin dato" #-- Gracias a este codigo solo quedarian 13 valores faltantes que se terminan completando y normalizando con el valor de "sin dato"



#---------------------------------------------------------------------------------------------------------------------------------------------
#APLICADO AL DATASET DE TEST

localilades_na_test = test["l2"][test.l3.isna()].drop_duplicates().values                                                                    
                                                                        
for i in localilades_na:
    if len(test["l3"][test["l2"] == i].mode()) > 0:
        test["l3"][(test["l2"] == i) & (test["l3"].isna())] = test["l3"][test["l2"] == i].mode()[0]
    
test["l3"][test["l3"].isna()] = "sin dato"


In [ ]:
#COMPLETADO DE COLUMNAS "LAT" Y "LON"


# Por ultimo vamos a completar las columnas "lat" y "lon" utilizando el promedio de los mismos registros que ahi en sus respectivas
# columnas teniendo en cuenta que este promedio de los faltantes vengan con respaldo de "l2" y "l3" para asegurar una mejor presicion.


lat_lon = train[["l2","l3"]][(train["lat"].isna()) & (train["lon"].isna())].drop_duplicates().values # Lista de "l2" y "l3" que estan en las filas donde no hay ni latitud ni longitud.

for i in lat_lon:
    print (i)
    prom_lon = train["lon"][(train["l2"]== i[0]) & (train["l3"]== i[1])].mean()
    prom_lat = train["lat"][(train["l2"]== i[0]) & (train["l3"]== i[1])].mean()
    print (prom_lon,prom_lat)
    train["lon"][(train["lon"].isna()) & (train["l2"]== i[0]) & (train["l3"]== i[1])] = prom_lon
    train["lat"][(train["lat"].isna()) & (train["l2"]== i[0]) & (train["l3"]== i[1])] = prom_lat

#-- Gracias a este codigo solo quedan 70 filas en longitud y latitud sin informacion. para mi modelo voy a utilizar el metodo de promedio en la columna de "lon" y "lat"
# respectivamente, para asi completar los valores faltantes con esa aproximacion.

train["lon"][train["lon"].isna()] = train["lon"].mean()
train["lat"][train["lat"].isna()] = train["lat"].mean()

train.drop(columns=["l2","l3"],inplace=True) 
#Al ser columnas con muchas variables categoricas las dropeo, de esa manera puedo ejecutar un modelo sencillo, con la mayoria de mis registros
#completados y columnas con registros numericos.



#---------------------------------------------------------------------------------------------------------------------------------------------
#APLICADO AL DATASET DE TEST

lat_lon_test = test[["l2","l3"]][(test["lat"].isna()) & (test["lon"].isna())].drop_duplicates().values

for i in lat_lon:
    print (i)
    prom_lon = test["lon"][(test["l2"]== i[0]) & (test["l3"]== i[1])].mean()
    prom_lat = test["lat"][(test["l2"]== i[0]) & (test["l3"]== i[1])].mean()
    print (prom_lon,prom_lat)
    test["lon"][(test["lon"].isna()) & (test["l2"]== i[0]) & (test["l3"]== i[1])] = prom_lon
    test["lat"][(test["lat"].isna()) & (test["l2"]== i[0]) & (test["l3"]== i[1])] = prom_lat

test["lon"][test["lon"].isna()] = test["lon"].mean()
test["lat"][test["lat"].isna()] = test["lat"].mean()

test.drop(columns=["l2","l3"],inplace=True) 


8) ONEHOTENCODER para mi columna categorica, y la creacion de la columna TARGET

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
#Utilizamos el metodo OneHotEncoder para poder modificar mi columna categorica "property_type" a una numerica.
codificador = OneHotEncoder()
cod_property_type = codificador.fit_transform(train[["property_type"]])
property_type = pd.DataFrame(cod_property_type.toarray(),columns=codificador.categories_)

train.reset_index(inplace=True) #-- Reseteo el indice para de esa manera no tener erroes en la concatenacion.


#Concateno las tablas train (tabla madre) con la tabla numerica de mi columna que era categorica "property_type"
train = pd.concat([train,property_type],axis="columns")
train.drop(columns="property_type",inplace=True) #-- Dropeo columna con valores categoricas que ya transforme con onehotencoder.
train.drop(columns="index",inplace=True) #-- Dropeo columna con indices que me genero pandas.



#Creo mi variable objetivo con la primicia de que si el precio esta por encima del promedio de la columa "price" es caro (1) caso contrario es barato (0)
train["target"] = 0
promedio = round(train["price"].mean(),2)
train["target"] = train["price"].apply(lambda x: 1 if x > promedio else 0)
train.drop(columns="price",inplace=True) #-- Dropeo columna de precio ya que para la evaluacion de mi modelo no es informacion util


#---------------------------------------------------------------------------------------------------------------------------------
#APLICADO AL DATASET DE TEST

codificador_t = OneHotEncoder()
cod_property_type_t = codificador_t.fit_transform(test[["property_type"]])
property_type_test = pd.DataFrame(cod_property_type_t.toarray(),columns=codificador_t.categories_)
test.reset_index(inplace=True)
test = pd.concat([test,property_type_test],axis="columns")
test.drop(columns="property_type",inplace=True) 
test.drop(columns="index",inplace=True) 

9) importamos los metodos correspondientes para la creacion de mi modelo de categorizacion.

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


#Decidi utilizar el modelo de regresion logisitica ya que no solo me dio buenos resultados luego de probarlo, sino que de 
#antemano pense en que si bien podia ser una solucion sencilla al promebla de la prediccion, encajaba bastante bien
#con el dataset final que transforme, dejando pocas columnas pero la mayoria de registros en estas intactos y completos.

In [17]:
X = train[train.columns[:-1]].to_numpy() #-- Creo mi set de datos para entrenar (sin mi variable objetivo)
Y = train[train.columns[-1]].to_numpy() #--  Creo mi set de entrenamiento sin mi variable objetivo


X_Train,X_Test,Y_Train,Y_Test = train_test_split(X,Y,test_size=0.30,random_state=77)
#Seleccionamos un 30% de datos para testear y el restante 70% para entrenar mi modelo

modelo=LogisticRegression(solver='lbfgs', max_iter=10000).fit(X_Train,Y_Train) #-- Entrenamos al modelo.
Y_predicho = modelo.predict(X_Test) #-- resultados de la prediccion de mi modelo

10) evaluacion de mi modelo con la importacion de las metricas para la matriz de confusion

In [19]:
from sklearn import metrics as metrics

In [20]:
metrics.recall_score(Y_Test,Y_predicho) # -- recall de 0.35
metrics.accuracy_score(Y_Test,Y_predicho) # -- accuracy_ de 0.80

0.34838662684827737

In [530]:
testeo = test.to_numpy()
resultado = modelo.predict(testeo)# -- prediccion con dataset de testeo

In [ ]:
df = pd.DataFrame(resultado)
df.rename(columns={0:"pred"},inplace=True)
df.to_csv("nicolasgallu.csv",index=False)

#archivo  "nicolasgallu" subido como resultados de las predicciones de mi modelo